In [32]:
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [33]:
#################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the table
Measurement = Base.classes.measurements
Station = Base.classes.stations

# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
#################################################
# Flask Setup
#################################################
app = Flask(__name__)


#################################################
# Flask Routes
#################################################

@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end>"
    )
@app.route("/api/v1.0/precipitation")
def precipitation():
    """Return available last year precipitation data for Honolulu"""
    # Query date and tobs
    results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >'2014-04-30').filter(Measurement.date <'2015-10-30').\
    order_by(Measurement.date.desc()).filter(Measurement.station =='USC00511918').all()


    # Convert list of tuples into normal list
    precipitation_data = list(np.ravel(results))

    return jsonify(precipitation_data)

@app.route("/api/v1.0/stations")
def station():
    """Return a list of all stations names"""
    # Query all station names
    results = session.query(Station.station, Station.name).group_by(Station.station).all()  


    # Convert list of tuples into normal list
    stations_data = list(np.ravel(results))

    return jsonify(stations_data)

@app.route("/api/v1.0/tobs")
def tobs():
    """Return a tobs for previous year"""
    # Query all tobs for last year
    results = session.query(Measurement.tobs).filter(Measurement.date <'2017-08-23').filter(Measurement.date >'2016-07-23').all()


    # Convert list of tuples into normal list
    tobs_data = list(np.ravel(results))

    return jsonify(tobs_data)


@app.route("/api/v1.0/<start>")
def start():
    """Return max.temp, avg.temp, min.temp for greater and equal to start date"""
    # Query all tobs for last year

    date_given = start.replace(dt.datetime(yyyy,mm, dd))
    for date in Measurement.date:
        startDate = date['start'].replace(dt.datetime(yyyy,mm, dd))
        
        if startDate==date_given:
            return jsonify(str(date))
    return jsonify({f"error":f"The data for the date(start) mentioned is not available"}),404

    results = session.query(func.max(Measurement.tobs), func.avg(Measurement.tobs), func.min(Measurement.tobs)).\
                           filter(Measurement.date <= startDate).all()
    temp_data = list(np.ravel(results))

    return jsonify(temp_data)



@app.route("/api/v1.0/<start>/<end>")
def start_end():
    """Return max.temp, avg.temp, min.temp between start date and end date"""
   


    date_given = start.replace(dt.strftime('%Y/%m/%d'))
    for date in Measurement.date:
        startDate = date['start'].replace(dt.strftime('%Y/%m/%d'))
        
        if startDate==date_given:
            return jsonify(str(date))
    return jsonify({f"error":f"The data for the date(start) mentioned is not available"}),404


    endDate_given = end.replace(dt.strftime('%Y/%m/%d'))
    for date in Measurement.date:
        endDate = date['end'].replace(dt.strftime('%Y/%m/%d'))
        
        if endDate==date_given:
            return jsonify(str(date))
    return jsonify({f"error":f"The data for the date(end) mentioned is not available"}),404

     # Query max, avg and min temps
    results = session.query(Measurement.tobs).filter(Measurement.date <'2017-08-23').filter(Measurement.date >'2016-07-23').all()
    
    tempr_data = list(np.ravel(results))

    return jsonify(tempr_data)



if __name__ == '__main__':
    app.run(debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Apr/2018 23:14:01] "GET / HTTP/1.1" 200 -
